<a href="https://colab.research.google.com/github/MaryamNourii/Knowledge-Tracing_Recommendation-system-/blob/main/RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/AI/Recommendation_system/KT1/KT1

/content/drive/MyDrive/AI/Recommendation_system/KT1/KT1


In [ ]:
questions = pd.read_csv("/content/drive/MyDrive/AI/Recommendation_system/questions.csv")
questions

,question_id,bundle_id,explanation_id,correct_answer,part,tags,deployed_at
0,q1,b1,e1,b,1,1;2;179;181,1558093217098
1,q2,b2,e2,a,1,15;2;182,1558093219720
2,q3,b3,e3,b,1,14;2;179;183,1558093222784
3,q4,b4,e4,b,1,9;2;179;184,1558093225357
4,q5,b5,e5,c,1,8;2;179;181,1558093228439
...,...,...,...,...,...,...,...
13164,q18139,b12202,e12202,b,2,24;26;183;182,1571733814684
13165,q18140,b12203,e12203,a,2,24;33;183;182,1571733815331
13166,q18141,b12204,e12204,a,2,24;26;183;182,1571733815951
13167,q18142,b12205,e12205,a,2,24;26;183;182,1571733816585


In [ ]:
ques_data = questions[['question_id', 'correct_answer']].copy()
ques_data

,question_id,correct_answer
0,q1,b
1,q2,a
2,q3,b
3,q4,b
4,q5,c
...,...,...
13164,q18139,b
13165,q18140,a
13166,q18141,a
13167,q18142,a


In [ ]:
path=r'/content/drive/MyDrive/AI/Recommendation_system/KT1/KT1'
d=[]
table_list=[]
s=pd.Series(os.listdir(path))
file_selected=s.sample(5000).to_numpy()
for file_name in file_selected:
    data_raw=pd.read_csv(path+'/'+file_name,encoding = "ISO-8859-15")
    data_raw['user_id']=pd.Series([file_name[:-4]]*len(data_raw))
    d.append([file_name[:-4],len(data_raw)])
    data=pd.DataFrame(data_raw,columns=['user_id']+data_raw.columns.to_list()[:-1])
    table_list.append(data)
df=pd.concat(table_list)
pd.set_option('display.max_rows',10)
df=df.reset_index(drop=True)
df

,user_id,timestamp,solving_id,question_id,user_answer,elapsed_time
0,u520858,1551506949787,1,q4637,c,17000
1,u520858,1551506974246,2,q1043,a,21000
2,u520858,1551507011616,3,q4025,d,33000
3,u520858,1551507056432,4,q5267,d,41000
4,u520858,1551507090784,5,q6056,c,30000
...,...,...,...,...,...,...
625204,u525660,1551856701173,4,q5805,a,30000
625205,u525660,1551856719975,5,q297,b,16000
625206,u525660,1551856741513,6,q6382,c,18000
625207,u328586,1542104600114,1,q5020,a,28000


In [ ]:
df.fillna('a',inplace=True)

In [ ]:
new_df = pd.merge(df, ques_data, on='question_id')
new_df

,user_id,timestamp,solving_id,question_id,user_answer,elapsed_time,correct_answer
0,u520858,1551506949787,1,q4637,c,17000,c
1,u620323,1561018029203,1,q4637,c,17000,c
2,u81481,1517909322596,1217,q4637,c,107000,c
3,u17485,1564279788917,298,q4637,c,30000,c
4,u521430,1561204800919,8058,q4637,c,8000,c
...,...,...,...,...,...,...,...
625204,u36001,1504259005805,166,q5140,a,10000,a
625205,u658608,1572499722000,1466,q17775,a,23000,a
625206,u658608,1572499848000,1468,q18128,b,17000,b
625207,u658608,1573447218000,1523,q17898,b,16000,b


In [ ]:
unique_users = new_df['user_id'].unique()
unique_questions = new_df['question_id'].unique()
unique_answers = new_df['correct_answer'].unique()

In [ ]:
unique_users

In [ ]:
new_df = df[df['user_id']=='u658608']
new_df.reset_index(inplace=True, drop=True)
new_df

In [ ]:
le = LabelEncoder()
new_df['problem_index'] = le.fit_transform(new_df['question_id'])
new_df['problem_index']

In [ ]:
num_skill = len(new_df['solving_id'].unique())
num_skill

In [ ]:
def convert_to_input_vector(qt, at, num_skills):
  input_vector = np.zeros(2*num_skill)
  output_vector = np.zeros(num_skill)

  skill_index = qt
  correct = at
  # if num_skill<100:
  if at:
    answer_index = skill_index
  else:
    answer_index = num_skill + skill_index

  input_vector[answer_index] = 1
  output_vector[skill_index] = at  # else:
  #     input_vector = np.random.normal(size=32)
  return input_vector,output_vector

In [ ]:
if num_skill <= 102:
    X = np.zeros((len(new_df), 2*num_skill))
    Y = np.zeros((len(new_df), num_skill))
    # for _, group in data.groupby('user_id'):
    current_sequence=[]
    for i, row in new_df.iterrows():
        input_vector,output_vector = convert_to_input_vector(row['solving_id'], row['correct_answer'], num_skill)
        X[i]= input_vector
        Y[i] = output_vector

In [ ]:
num_problems=len(data)
num_problems

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.001, random_state=42)

X_train=X_train.reshape((X_train.shape[0],1, X_train.shape[1]))
y_train=y_train.reshape((y_train.shape[0],1, y_train.shape[1]))
X_test=X_test.reshape((X_test.shape[0],1, X_test.shape[1]))
y_test=y_test.reshape((y_test.shape[0],1, y_test.shape[1]))
X_val=X_val.reshape((X_val.shape[0],1, X_val.shape[1]))
y_val=y_val.reshape((y_val.shape[0],1, y_val.shape[1]))
y_train.shape,X_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

In [ ]:
inputs = Input(shape=(None,2*num_skill))
lstm1 = LSTM(128, return_sequences=True)(inputs)
output = Dense(num_skill, activation='sigmoid')(lstm1)
model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=32, epochs=100, validation_data=(X_val, y_val),shuffle=True)

In [ ]:
model.save('dkt_model.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_sequences, np.array(test_labels))
print('Test accuracy:', test_acc)